In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Column names for question items
ori_responses = ["LOY_1", "LOY_2", "LOY_3"]
target_df = pd.read_excel('DataInBrief_Bankdata.xlsx')
selected_columns_df = target_df[ori_responses]
base_path = './Data_Case1' ## example path for given result data we provided


In [3]:
distance_dict = {
    1: -1, 
    2: -1, 
    3: -1, 
    4: 0, 
    5: 1, 
    6: 1, 
    7: 1
}

In [4]:
results = []

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        llm_file = os.path.join(folder_path, 'llm_responses.csv')
        llm_df = pd.read_csv(llm_file, index_col=0)
        print(f"==========Results of {folder}===========\n")

        original_df_mapped = selected_columns_df.applymap(lambda x: distance_dict.get(x, None))
        llm_df_mapped = llm_df.applymap(lambda x: distance_dict.get(x, None))
        if list(original_df_mapped.index) != list(llm_df_mapped.index):
            print("Error!")
        accuracies = []
        for i in range(1, 4):
            llm_df_mapped[f'Q{i}_original'] = original_df_mapped.iloc[:, i - 1]
            llm_df_mapped[f'Q{i}_check'] = np.where(llm_df_mapped[f'Q{i}_original'] == llm_df_mapped[f'Q{i}'], 1, 0)
            accuracy = llm_df_mapped[f'Q{i}_check'].mean() * 100
            accuracies.append(accuracy)
        mean_accuracy = sum(accuracies) / len(accuracies)
        
        print(f"Q1 Accuracy: {accuracies[0]:.2f}%")
        print(f"Q2 Accuracy: {accuracies[1]:.2f}%")
        print(f"Q3 Accuracy: {accuracies[2]:.2f}%")
        print(f"Mean Accuracy: {mean_accuracy:.2f}%\n")

        results.append({
            'Folder': folder,
            'Q1_Accuracy': accuracies[0],
            'Q2_Accuracy': accuracies[1],
            'Q3_Accuracy': accuracies[2],
            'Mean_Accuracy': mean_accuracy
        })

==========Results of Demo_prompting===========

Q1 Accuracy: 83.11%
Q2 Accuracy: 30.67%
Q3 Accuracy: 40.74%
Mean Accuracy: 51.51%

==========Results of Omni_prompting===========

Q1 Accuracy: 84.89%
Q2 Accuracy: 65.19%
Q3 Accuracy: 43.56%
Mean Accuracy: 64.54%



In [5]:
save_path = './analysis_result/appendix/consistency_analysis_case1/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

results_df = pd.DataFrame(results)
results_df.to_csv('./analysis_result/appendix/consistency_analysis_case1/result.csv', index=False)